In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, ward_tree, SpectralClustering

# ustawianie statyczne ziarna losowania ->
# zapewnienie powtarzalności generowanychn wyników
np.random.seed(1234)

In [4]:
clean_df2 = pd.read_csv('clean_df2.csv')
clean_df2.shape

(45211, 97)

In [5]:
input_df = np.array(clean_df2)[:, 1:]
input_df.shape

(45211, 96)

In [6]:
models = [
    ('KM_2', KMeans(n_clusters=2)),
    ('KM_3', KMeans(n_clusters=3)),
    ('KM_4', KMeans(n_clusters=4)),
    ('KM_5', KMeans(n_clusters=5)),
    ('KM_6', KMeans(n_clusters=6)),
]

results = {}
for name, model in models:
    clf = model
    results[name] = clf.fit_predict(input_df)
    print(name, ' results:')
    print(pd.Series(results[name]).value_counts())
    print('\n')


KM_2  results:
0    31623
1    13588
dtype: int64


KM_3  results:
2    20015
1    13532
0    11664
dtype: int64


KM_4  results:
1    14453
2    13543
0    10013
3     7202
dtype: int64


KM_5  results:
3    14130
1    10016
0     7503
4     7199
2     6363
dtype: int64


KM_6  results:
0    13308
1     8701
3     7149
2     6778
4     4861
5     4414
dtype: int64




In [7]:
pd.DataFrame(results).to_csv('results.csv')

In [8]:
results_df = pd.DataFrame(results)

In [9]:
original_df = pd.read_csv('./data/bank/bank-full.csv', 
                 sep=';',
                 dtype={'age': 'int64',
                        'job': 'category',
                        'marital': 'category',
                        'education': 'category',
                        'default': 'category',
                        'balance': 'int64',
                        'housing': 'category',
                        'loan': 'category',
                        'contact': 'category',
                        'day': 'int64',
                        'month': 'category',
                        'duration': 'int64',
                        'campaign': 'int64',
                        'pdays': 'int64',
                        'previous': 'int64',
                        'poutcome': 'category',
                        'y': 'category'})

combined_df = pd.concat([clean_df2, original_df, results_df], axis=1)

In [10]:
combined_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,campaign,pdays,previous,poutcome,y,KM_2,KM_3,KM_4,KM_5,KM_6
0,0,0,0,0,0,0,0,0,0,0,...,1,-1,0,unknown,no,1,1,2,4,1
1,1,0,0,0,0,0,0,0,0,0,...,1,-1,0,unknown,no,0,0,0,1,3
2,2,0,0,0,0,0,0,0,0,0,...,1,-1,0,unknown,no,0,2,1,3,0
3,3,0,0,0,0,0,0,0,0,0,...,1,-1,0,unknown,no,0,2,3,0,5
4,4,0,0,0,0,0,0,0,0,0,...,1,-1,0,unknown,no,0,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,45206,0,0,0,0,0,0,0,0,0,...,3,-1,0,unknown,yes,1,1,2,4,1
45207,45207,0,0,0,0,0,0,0,0,0,...,2,-1,0,unknown,yes,0,0,3,0,2
45208,45208,0,0,0,0,0,0,0,0,0,...,5,184,3,success,yes,0,2,1,3,0
45209,45209,0,0,0,0,0,0,0,0,0,...,4,-1,0,unknown,no,0,2,1,3,5


In [11]:
combined_df['total'] = 1

combined_df['age_breaks'] = pd.cut(combined_df['age'], 
                                  bins=[17, 25, 35, 45, 55, 65, 75, 85, 90, 100]).astype(str)
combined_df['age_breaks']

0        (55, 65]
1        (35, 45]
2        (25, 35]
3        (45, 55]
4        (25, 35]
           ...   
45206    (45, 55]
45207    (65, 75]
45208    (65, 75]
45209    (55, 65]
45210    (35, 45]
Name: age_breaks, Length: 45211, dtype: object

In [13]:
# przecięcie powstałych klastrów przez zmienne wzięte do analizy (age, job, marital, education)

In [14]:
def pivot_table_KM(index, column):
    return pd.pivot_table(combined_df, 
               values='total', 
               index=index, 
               columns=[column], 
               aggfunc=np.sum,
               margins=True,
               margins_name='Total',
               fill_value=0).T
    
#pivot_table_KM('KM_3', 'age_breaks')

In [15]:
for table in [pivot_table_KM('KM_2', 'age_breaks'),
pivot_table_KM('KM_2', 'job'),
pivot_table_KM('KM_2', 'marital'),
pivot_table_KM('KM_2', 'education')]:
    print(table, '\n\n')

KM_2            0      1  Total
age_breaks                     
(17, 25]     1084    252   1336
(25, 35]     9928   5643  15571
(35, 45]     9739   4117  13856
(45, 55]     6983   2565   9548
(55, 65]     3041   1108   4149
(65, 75]      372    118    490
(75, 85]      201     33    234
(85, 90]       19      1     20
(90, 100]       7      0      7
Total       31374  13837  45211 


KM_2               0      1  Total
job                               
admin.          4599    572   5171
blue-collar     9583    149   9732
entrepreneur     801    686   1487
housemaid       1067    173   1240
management      1121   8337   9458
retired         1898    366   2264
self-employed    746    833   1579
services        3952    202   4154
student          715    223    938
technician      5629   1968   7597
unemployed      1014    289   1303
unknown          249     39    288
Total          31374  13837  45211 


KM_2          0      1  Total
marital                      
divorced   3679   1528   

In [16]:
for table in [pivot_table_KM('KM_3', 'age_breaks'),
pivot_table_KM('KM_3', 'job'),
pivot_table_KM('KM_3', 'marital'),
pivot_table_KM('KM_3', 'education')]:
    print(table, '\n\n')

KM_3            0      1      2  Total
age_breaks                            
(17, 25]      164    926    246   1336
(25, 35]     4890   5089   5592  15571
(35, 45]     6713   3174   3969  13856
(45, 55]     5457   1695   2396   9548
(55, 65]     2492    645   1012   4149
(65, 75]      300     76    114    490
(75, 85]      143     63     28    234
(85, 90]       12      7      1     20
(90, 100]       5      2      0      7
Total       20176  11677  13358  45211 


KM_3               0      1      2  Total
job                                      
admin.          2451   2148    572   5171
blue-collar     6908   2675    149   9732
entrepreneur     617    184    686   1487
housemaid        798    269    173   1240
management      1100    500   7858   9458
retired         1477    421    366   2264
self-employed    543    203    833   1579
services        2316   1636    202   4154
student           32    683    223    938
technician      3155   2474   1968   7597
unemployed       601    4

In [17]:
for table in [pivot_table_KM('KM_4', 'age_breaks'),
pivot_table_KM('KM_4', 'job'),
pivot_table_KM('KM_4', 'marital'),
pivot_table_KM('KM_4', 'education')]:
    print(table, '\n\n')

KM_4            0      1     2      3  Total
age_breaks                                  
(17, 25]      251    848   100    137   1336
(25, 35]     5624   4637  1320   3990  15571
(35, 45]     4030   2709  2311   4806  13856
(45, 55]     2456   1321  2200   3571   9548
(55, 65]     1031    442  1236   1440   4149
(65, 75]      118     39   202    131    490
(75, 85]       32     14   140     48    234
(85, 90]        1      3    13      3     20
(90, 100]       0      1     5      1      7
Total       13543  10014  7527  14127  45211 


KM_4               0      1     2      3  Total
job                                            
admin.           572   2088   209   2302   5171
blue-collar      149   1798  4113   3672   9732
entrepreneur     686    157   235    409   1487
housemaid        173    137   660    270   1240
management      8043    431   294    690   9458
retired          366    228   896    774   2264
self-employed    833    185   150    411   1579
services         202   15

In [18]:
for table in [pivot_table_KM('KM_5', 'age_breaks'),
pivot_table_KM('KM_5', 'job'),
pivot_table_KM('KM_5', 'marital'),
pivot_table_KM('KM_5', 'education')]:
    print(table, '\n\n')

KM_5            0      1     2     3     4  Total
age_breaks                                       
(17, 25]      848    141    96   218    33   1336
(25, 35]     4637   4000  1306  3470  2158  15571
(35, 45]     2709   4859  2248  1561  2479  13856
(45, 55]     1321   3671  2096   803  1657   9548
(55, 65]      442   1545  1130   283   749   4149
(65, 75]       39    162   171    24    94    490
(75, 85]       14     67   121     4    28    234
(85, 90]        3      3    13     0     1     20
(90, 100]       1      5     1     0     0      7
Total       10014  14453  7182  6363  7199  45211 


KM_5               0      1     2     3     4  Total
job                                                 
admin.          2088   2302   209   330   242   5171
blue-collar     1798   3672  4113    89    60   9732
entrepreneur     157    461   183   233   453   1487
housemaid        137    303   627    59   114   1240
management       431    690   275  3601  4461   9458
retired          228    87

In [19]:
for table in [pivot_table_KM('KM_6', 'age_breaks'),
pivot_table_KM('KM_6', 'job'),
pivot_table_KM('KM_6', 'marital'),
pivot_table_KM('KM_6', 'education')]:
    print(table, '\n\n')

KM_6            0     1     2     3     4     5  Total
age_breaks                                            
(17, 25]       85   843    35   216    61    96   1336
(25, 35]     3217  4096  2403  3225  1400  1230  15571
(35, 45]     4422  1647  2986  1054  1656  2091  13856
(45, 55]     3644   452  2167   293  1025  1967   9548
(55, 65]     1656   104   972    60   269  1088   4149
(65, 75]      192     6   105    13     3   171    490
(75, 85]       81     0    32     0     0   121    234
(85, 90]        5     1     1     0     0    13     20
(90, 100]       6     0     0     0     0     1      7
Total       13308  7149  8701  4861  4414  6778  45211 


KM_6               0     1     2     3     4     5  Total
job                                                      
admin.          2968  1452   296   276     0   179   5171
blue-collar        0  1411    64    85  4414  3758   9732
entrepreneur     534    84   550   136     0   183   1487
housemaid        379    61   132    41     0   

In [15]:
# Aglomerative clustering on reduced df

ac_df = input_df[::10]
ac_df.shape

ag_models = [
    ('AG_2', AgglomerativeClustering(n_clusters=2)),
    ('AG_3', AgglomerativeClustering(n_clusters=3)),
    ('AG_4', AgglomerativeClustering(n_clusters=4)),
    ('AG_5', AgglomerativeClustering(n_clusters=5)),
    ('AG_6', AgglomerativeClustering(n_clusters=6))
]

ag_results = {}

for name, model in ag_models:
    clf = model
    ag_results[name] = clf.fit_predict(ac_df)
    print(name, ' results:')
    print(pd.Series(ag_results[name]).value_counts())
    print('\n')

AG_2  results:
0    3195
1    1327
dtype: int64


AG_3  results:
0    2294
1    1327
2     901
dtype: int64


AG_4  results:
3    1352
0    1327
1     942
2     901
dtype: int64


AG_5  results:
3    1352
0     942
2     901
4     706
1     621
dtype: int64


AG_6  results:
3    1352
0     901
4     706
2     651
1     621
5     291
dtype: int64




In [16]:
ag_results

{'AG_2': array([1, 0, 0, ..., 0, 0, 0], dtype=int32),
 'AG_3': array([1, 0, 0, ..., 0, 0, 0], dtype=int32),
 'AG_4': array([0, 1, 3, ..., 3, 3, 3], dtype=int32),
 'AG_5': array([4, 0, 3, ..., 3, 3, 3], dtype=int32),
 'AG_6': array([4, 5, 3, ..., 3, 3, 3], dtype=int32)}

In [21]:
# SpectralClustering

# sc_models = [
#     ('SC_3', SpectralClustering(n_clusters=3))
# ]

# sc_results = {}

# for name, model in sc_models:
#     clf = model
#     sc_results[name] = clf.fit_predict(ac_df)
#     print(name, ' results:')
#     print(pd.Series(ag_results[name]).value_counts())
#     print('\n')